In [1]:
#importing libraries
import json
import os

#creating path
path = '/kaggle/input/sample-json/pos_0.png.json'

#storing filename
filename = path .split('/')[-1]

#storing filetype
if "png" in filename:
    image_type = "image"

#loading the data
with open(path) as f:
    data = json.load(f)

#checking data
data

{'description': '',
 'tags': [],
 'size': {'height': 720, 'width': 1280},
 'objects': [{'id': 28602834,
   'classId': 17389,
   'description': '',
   'geometryType': 'rectangle',
   'labelerLogin': 'muyeed',
   'createdAt': '2021-11-24T04:04:09.702Z',
   'updatedAt': '2021-11-24T04:04:09.702Z',
   'tags': [{'id': 29485103,
     'tagId': 32292,
     'name': 'Type',
     'value': 'SUV',
     'labelerLogin': 'muyeed',
     'createdAt': '2021-11-24T04:04:09.702Z',
     'updatedAt': '2021-11-24T04:04:09.702Z'},
    {'id': 29485102,
     'tagId': 32295,
     'name': 'Pose',
     'value': 'Rear',
     'labelerLogin': 'muyeed',
     'createdAt': '2021-11-24T04:04:09.702Z',
     'updatedAt': '2021-11-24T04:04:09.702Z'},
    {'id': 29485101,
     'tagId': 32297,
     'name': 'Model',
     'value': 'UN',
     'labelerLogin': 'muyeed',
     'createdAt': '2021-11-24T04:04:09.702Z',
     'updatedAt': '2021-11-24T04:04:09.702Z'},
    {'id': 29485100,
     'tagId': 32296,
     'name': 'Make',
     'va

In [2]:
#extracting info from the json

#intializing necessary dictionary and lists
new_name={}
bbox=[]
bbox_vehicle=[]
bbox_license=[]
occlusion=[]

#grabbing info from the JSON
for obj in data['objects']:
    for attr in obj['tags']:
        new_name[attr['name']] = attr['value']
    for x in (obj['points']['interior']):
        occlusion.append(x)
    for x in (obj['points']['exterior']):
        bbox.append(x)
if len(bbox) != 0:
    new_name['bbox_vehicle'] = bbox[0] + bbox[1]
    
    if len(bbox[0] + bbox[1])>0:
        new_name['presence_v'] = 1
    else :
        new_name['presence_v'] = 0
    
    if len(bbox) > 2:
        new_name['bbox_license'] = bbox[2] + bbox[3]
        
        if len(bbox[2] + bbox[3])>0:
            new_name['presence_l'] = 1
        else :
            new_name['presence_l'] = 0
        
new_name['Occlusion'] = len(occlusion)

#checking extracting info
new_name

{'Type': 'SUV',
 'Pose': 'Rear',
 'Model': 'UN',
 'Make': 'Lexus',
 'Color': 'Silver',
 'Difficulty Score': '0',
 'Value': 'CA, 7LVS119',
 'bbox_vehicle': [396, 3, 773, 314],
 'presence_v': 1,
 'bbox_license': [535, 158, 622, 213],
 'presence_l': 1,
 'Occlusion': 0}

In [3]:
#tackling missing attributes or objects
list_attr = ["Type","Pose","Model","Make","Color","Difficulty Score","Value","Occlusion"]
list_attr_pr = ["presence_v","presence_l"]
list_attr_bb = ["bbox_vehicle","bbox_license"]

for x in list_attr:
    if x not in new_name:
        new_name[x]=None

for x in list_attr_pr:
    if x not in new_name:
        new_name[x]=0

for x in list_attr_bb:
    if x not in new_name:
        new_name[x]=[]
    
new_name

{'Type': 'SUV',
 'Pose': 'Rear',
 'Model': 'UN',
 'Make': 'Lexus',
 'Color': 'Silver',
 'Difficulty Score': '0',
 'Value': 'CA, 7LVS119',
 'bbox_vehicle': [396, 3, 773, 314],
 'presence_v': 1,
 'bbox_license': [535, 158, 622, 213],
 'presence_l': 1,
 'Occlusion': 0}

In [4]:
#creating formatted json

formatted_json={}


formatted_json["dataset_name"] = filename
formatted_json["image_link"] = ""
formatted_json["annotation_type"] = image_type
formatted_json["annotation_objects"] = {"vehicle":{"presence": new_name["presence_v"],
                                                  "bbox": new_name["bbox_vehicle"]},
                                     "license_plate":{
                                         "presence": new_name["presence_l"],
                                         "bbox": new_name["bbox_license"]}}
formatted_json["annotation_attributes"] = {"vehicle":{
                                                      "Type":new_name["Type"],
                                                      "Pose": new_name["Pose"],
                                                      "Model": new_name["Model"],
                                                      "Make": new_name["Make"],
                                                      "Color":new_name["Color"]
                                                       },
                                           "license_plate":{
                                               "Difficulty Score":new_name["Difficulty Score"],
                                               "Value":new_name["Value"],
                                               "Occlusion":new_name["Occlusion"]
                                           }}

formatted_json = [formatted_json]

s = json.dumps(formatted_json,indent=4)

print(s)

[
    {
        "dataset_name": "pos_0.png.json",
        "image_link": "",
        "annotation_type": "image",
        "annotation_objects": {
            "vehicle": {
                "presence": 1,
                "bbox": [
                    396,
                    3,
                    773,
                    314
                ]
            },
            "license_plate": {
                "presence": 1,
                "bbox": [
                    535,
                    158,
                    622,
                    213
                ]
            }
        },
        "annotation_attributes": {
            "vehicle": {
                "Type": "SUV",
                "Pose": "Rear",
                "Model": "UN",
                "Make": "Lexus",
                "Color": "Silver"
            },
            "license_plate": {
                "Difficulty Score": "0",
                "Value": "CA, 7LVS119",
                "Occlusion": 0
            }
        }
    }
]


In [5]:
with open("formatted_"+ filename,"w") as d:
    d.write(s)